In [1]:
# Random Search для RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.stats import randint, uniform
import numpy as np

# 1) Дані (data)
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2) Модель (model)
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# 3) Простір гіперпараметрів (hyperparameter search space)
param_distributions = {
    "n_estimators": randint(100, 800),          # кількість дерев (number of trees)
    "max_depth": randint(3, 30),                # глибина дерева (tree depth)
    "min_samples_split": randint(2, 20),
    "min_samples_leaf": randint(1, 10),
    "max_features": uniform(0.3, 0.7),          # частка ознак (feature fraction)
    "bootstrap": [True, False]
}

# 4) Random Search (випадковий пошук)
rnd_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_distributions,
    n_iter=40,                 # скільки випадкових комбінацій перевірити (number of sampled configs)
    scoring="f1",              # метрика оптимізації (optimization metric)
    cv=5,                      # к-ть фолдів CV (folds)
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# 5) Навчання (fit) та оцінювання (evaluate)
rnd_search.fit(X_train, y_train)
print("Найкращі параметри (best params):", rnd_search.best_params_)
print("CV f1 найкращої моделі (best score):", rnd_search.best_score_)

best_model = rnd_search.best_estimator_
y_pred = best_model.predict(X_test)
print("\nЗвіт по тесту (test classification report):\n")
print(classification_report(y_test, y_pred, digits=3))


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Найкращі параметри (best params): {'bootstrap': False, 'max_depth': 27, 'max_features': 0.3325152889039984, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 120}
CV f1 найкращої моделі (best score): 0.9703095256882837

Звіт по тесту (test classification report):

              precision    recall  f1-score   support

           0      0.951     0.929     0.940        42
           1      0.959     0.972     0.966        72

    accuracy                          0.956       114
   macro avg      0.955     0.950     0.953       114
weighted avg      0.956     0.956     0.956       114



In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 1) Дані
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2) Модель
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# 3) Ґратка параметрів (усі варіанти будуть перевірені)
param_grid = {
    "n_estimators": [200, 400, 600],
    "max_depth": [None, 8, 12, 16],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [0.5, 0.7, 1.0],
    "bootstrap": [True]  # фіксуємо, щоб не роздувати комбінації
}

# 4) CV та пошук
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring="f1",      # метрика (може бути 'accuracy', 'roc_auc', 'f1_macro' тощо)
    cv=cv,
    n_jobs=-1,
    verbose=1,
    refit=True         # автоматично перевчить найкращу конфігурацію на всіх train даних CV
)
grid.fit(X_train, y_train)

print("Найкращі параметри:", grid.best_params_)
print("CV f1 найкращої моделі:", grid.best_score_)

# 5) Оцінка на тесті
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
print("\nЗвіт по тесту:\n", classification_report(y_test, y_pred, digits=3))


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Найкращі параметри: {'bootstrap': True, 'max_depth': None, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
CV f1 найкращої моделі: 0.9718611825395602

Звіт по тесту:
               precision    recall  f1-score   support

           0      0.951     0.929     0.940        42
           1      0.959     0.972     0.966        72

    accuracy                          0.956       114
   macro avg      0.955     0.950     0.953       114
weighted avg      0.956     0.956     0.956       114

